# 유튜브 크롤링 후 DB 적재

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [2]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기 때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울'

'침대', '매트리스', '토퍼', '소파', '테이블', '식탁', '책상', '거실장', 'tv장', '서랍', '수납장', 

keyword_list = ['진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '콘솔','거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버', '블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어']

In [170]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
keyword = '진열장'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)

#사이트 최대화
driver.maximize_window()

# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)

# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

# 업로드 날짜 -> 올해 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

In [171]:
# 제목 & 링크 크롤링

# 스크롤 다운 50번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,50):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

print('영상 개수: ', len(name_list))

영상 개수:  154


In [159]:
# 업로드일 & 해시태그 가져오기
day_list = [] #업로드일
hashtag_list = [] #해시태그

for i in range(0,len(name_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(2)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
        get_hashtag(outline_test)

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(2)  

        #업로드일 정보 가져오기
        day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        #day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
        get_hashtag(outline_test)

print('끝')

1 : 일반 2021. 9. 28.
#마켓비 #수납장 #화이트수납장 #틈새수납장 #거실수납장 #베란다수납장 #미닫이수납장 #다용도수납장 #틈새수납 #옷수납장 #낮은수납장 #높은수납장 #슬림수납장 #마켓비수납장 #수납 #수납정리 #랜선집들이 #온라인집들이 #화이트인테리어 #집꾸미기 #방꾸미기 #내방꾸미기 #홈스타일링 #홈스타그램 #홈데코 #신혼집인테리어 #원룸인테리어
2 : 일반 2021. 8. 4.
#오늘의집 #방꾸미기 #책상꾸미기
3 : 일반 2022. 2. 25.
#이케아 #이케아 #이케아캐비넷 #이케아 #마켓비 #이케아 #이케아 #후리몰
4 : 일반 2021. 7. 11.

5 : 일반 2022. 5. 31.
#신혼브이로그 #부부브이로그 #일상브이로그
6 : 일반 2022. 2. 27.
#주방수납장 #주방수납장추천 #주방수납장추천순위 #인기주방수납장
7 : 일반 2021. 10. 4.

8 : 일반 2022. 2. 7.
#가구레시피 #가구레시피 #가구레시피 #가구레시피 #동서가구 #가온리빙 #룸앤홈 #가구레시피 #데코라인
9 : 일반 2021. 6. 12.

10 : 일반 2021. 12. 2.
#전신거울 #수납장 #옷장 #화장대 #원룸 #좁은방 #인테리어
11 : 일반 2022. 2. 17.
#홈앤미 #달리빙앤 #공간사이 #제우스가구 #보니애가구 #와이드 #가이온 #서진퍼니처 #내일의집
12 : 일반 2022. 4. 28.
#마켓비캐비넷 #철제수납장 #외부신발장
13 : 일반 2022. 2. 13.
#다이소주방정리 #싱크대정리 #다이소추천템
14 : 일반 2021. 12. 5.

15 : 일반 2022. 3. 22.
#인테리어소품 #인테리어 #집꾸미기 #마켓비 #마켓비인테리어 #셀프인테리어 #가구추천 #마켓비프라이빗
16 : 일반 2022. 2. 18.
#홈앤미 #준우드 #공간사이 #달리빙앤 #와이드 #제우스가구 #가이온 #내일의집
17 : 일반 2022. 3. 12.

18 : 일반 2021. 9. 10.
#이케아 #이케아조립
19 : 일반 2021.

In [168]:
day_list2 = [] #전처리 후 업로드일

#업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

# datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

# 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

youtubeDf.tail()

,제목,업로드일,주소,해시태그
147,1분안에 냉장고틈새수납장 가성비 상품 추천순위 top10 정리,2022-03-25,https://www.youtube.com/watch?v=_FjAr4WN7Uw,#엣지퍼니처 #랩소디 #돌로미티 #냉장고 #잉글랜더 #데이앤데이 #돌로미티 #리빙숲...
148,[단독] 유아 수납장서 카드뮴 나왔는데…한 색깔만 리콜 / SBS,2021-06-09,https://www.youtube.com/watch?v=rBMIyCD8UwE,#수납장
149,"갬성갬성 묻어나는 나만의 DIY 선반 수납장,, 내 집에 딱 맞는 거다..이거다",2022-05-03,https://www.youtube.com/watch?v=93QG07wba6U,#오늘의집 #혼살족 #수납장 #원룸인테리어
150,"광고# 가성비 거실수납장 가격,성능,후기 추천순위 top10 정리",2022-05-04,https://www.youtube.com/watch?v=0EQJvKYzHmQ,#홈앤미 #라자가구 #홈앤미 #동서가구 #서진퍼니처 #서진퍼니처 #홈앤미 #제이디퍼...
151,인기제품추천 높은수납장 비교 추천순위 top10 정리,2022-04-15,https://www.youtube.com/watch?v=NLe5CsqtS2A,#라자가구 #라자가구 #라자가구 #가이온 #가이온


In [169]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_interior'

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()


In [167]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

#print(hashtag_list[130][:207])

0 185
130 205


In [166]:
hashtag_list[130] = hashtag_list[130][:205]

# #############여기까지만#############

In [ ]:
def crawling_yt(keyword) :
    warnings.filterwarnings(action='ignore')

    driver = webdriver.Chrome('C:\chromedriver.exe')

    # 키워드를 검색한 유튜브 주소 접속
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
    driver.get(url)

    #사이트 최대화
    driver.maximize_window()

    # 필터 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
    time.sleep(1)

    # 구분 -> 동영상 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

    # 업로드 날짜 -> 올해 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

    # 스크롤 다운 50번 실행
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)

    for i in range(1,50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.select('a#video-title')
    video_url = soup.select('a#video-title')

    video_num_list = [] #영상번호
    name_list = [] #영상제목
    url_list = [] #영상주소

    for i in range(len(name)):
        video_num_list.append(i+1)
        name_list.append(name[i].text.strip())

    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

    print('영상 개수: ', len(name_list))

    # 업로드일 & 해시태그 가져오기
    day_list = [] #업로드일
    hashtag_list = [] #해시태그

    for i in range(0,len(name_list)):
        
        if url_list[i].split('/')[3] == 'shorts':  #쇼츠

            print(i+1, ': 쇼츠', end=' ')

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            
            driver.get(url_list[i])

            time.sleep(2)

            #더보기 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
            time.sleep(2)

            #설명 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
            time.sleep(2)

            #업로드일 정보 가져오기
            day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

            print(day)
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
            get_hashtag(outline_test)

        else :
            print(i+1, ': 일반', end=' ')

            driver.get(url_list[i])
                
            time.sleep(2)    

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            time.sleep(2)  

            #업로드일 정보 가져오기
            #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
            day = driver.find_element_by_id("info-strings").text.split('\n')[0]
            
            print(day)
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
            get_hashtag(outline_test)

    day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
    for day in day_list : 
        day = day.replace('.', '')
        day = day.replace('최초 공개: ', '')
        day = day.replace('실시간 스트리밍 시작일: ', '')
        day = day.replace(' ', '-')
        day_list2.append(day)

    return name_list, day_list2, url_list, hashtag_list


In [ ]:


#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

# datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

# 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

youtubeDf.tail()

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_clothes'

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()
